# Digitoner

The purpose of this notebook is to collect notes and code on the reverse engineering of the Digitone's sysex file structure. So far, I have been able to decode the patch sysex message dump somewhat reliably. Further research is necessary to decode the pattern and project dumps. 

The main goal of this project is to take the tagged data from the digitone's patch dump, separate them into individual patches, and use the tagging data to train a neural net that will generate a patch from a desired tag. The secondary goal of this project is to create a python library that will allow others to use the information I uncover for their own projects.

### Acknowledgements

I would like to recognize the good people in [this elektronauts thread](https://www.elektronauts.com/t/decoding-the-digitone-sysex/62731) who helped point me in the correct direction. Finally, the [libanalogrytm](https://github.com/bsp2/libanalogrytm) was a huge help in understanding the elektron sysex and their code was a major guidance in this development.

## Getting started

First thing we'll do is load the necessary libraries, as well as setting up some of our constants. `SYSEX_FILE` is not truly a constant but should be used whenever a new dataset is brought into the notebook. `TAGS` represents all the possible tags on the digitone in the order of which they appear. `PARAMETERS` is all of the parameters in each patch on the digitone. 

In [671]:
import os
from binascii import hexlify, unhexlify

SYSEX_FILE = 'parameters.syx'

TAGS = (
        'KICK', 'SNAR', 'DEEP', 'BRAS', 'STRI', 'PERC', 'HHAT', 'CYMB', 'EVOL', 'EXPR',
        'BASS', 'LEAD', 'PAD', 'TXTR', 'CRD', 'SFX', 'ARP', 'METL', 'ACOU', 'ATMO', 
        'NOIS', 'GLCH', 'HARD', 'SOFT', 'DARK', 'BRGT', 'UNTG', 'EPIC', 'FAIL', 'LOOP',
        'MINE', 'FAV'
       )

TAG_LOOKUP = {  '0': {
                      b'1': [TAGS[23]], 
                      b'2': [TAGS[31]], 
                      b'3': [TAGS[23], TAGS[31]]
                      }, 
                '1': {
                      b'4': [TAGS[7]], 
                      b'8': [TAGS[15]], 
                      b'c': [TAGS[7], TAGS[15]]
                      },
                '2': {
                      b'1': [TAGS[28]],
                      b'2': [TAGS[29]],
                      b'3': [TAGS[28], TAGS[29]],
                      b'4': [TAGS[30]],
                      b'5': [TAGS[30], TAGS[28]],
                      b'6': [TAGS[29], TAGS[30]],
                      b'7': [TAGS[28], TAGS[29], TAGS[30]]
                     },
                '3': {
                      b'1': [TAGS[24]],
                      b'2': [TAGS[25]],
                      b'3': [TAGS[24], TAGS[25]],
                      b'4': [TAGS[26]],
                      b'5': [TAGS[24], TAGS[26]],
                      b'6': [TAGS[25], TAGS[26]],
                      b'7': [TAGS[24], TAGS[25], TAGS[26]],
                      b'8': [TAGS[27]],
                      b'9': [TAGS[24], TAGS[27]],
                      b'a': [TAGS[25], TAGS[27]],
                      b'b': [TAGS[24], TAGS[25], TAGS[27]],
                      b'c': [TAGS[26], TAGS[27]],
                      b'd': [TAGS[24], TAGS[25], TAGS[27]],
                      b'e': [TAGS[24], TAGS[26], TAGS[27]],
                      b'f': [TAGS[24], TAGS[25], TAGS[26], TAGS[27]]
                     },
                '4': {
                      b'1': [TAGS[20]],
                      b'2': [TAGS[21]],
                      b'3': [TAGS[20], TAGS[21]],
                      b'4': [TAGS[22]],
                      b'5': [TAGS[20], TAGS[22]],
                      b'6': [TAGS[21], TAGS[22]],
                      b'7': [TAGS[20], TAGS[21], TAGS[22]]
                     },
                '5': {
                      b'1': [TAGS[16]],
                      b'2': [TAGS[17]],
                      b'3': [TAGS[16], TAGS[17]],
                      b'4': [TAGS[18]],
                      b'5': [TAGS[16], TAGS[18]],
                      b'6': [TAGS[25], TAGS[26]],
                      b'7': [TAGS[16], TAGS[17], TAGS[18]],
                      b'8': [TAGS[19]],
                      b'9': [TAGS[16], TAGS[19]],
                      b'a': [TAGS[17], TAGS[19]],
                      b'b': [TAGS[16], TAGS[17], TAGS[19]],
                      b'c': [TAGS[18], TAGS[19]],
                      b'd': [TAGS[16], TAGS[17], TAGS[19]],
                      b'e': [TAGS[16], TAGS[18], TAGS[19]],
                      b'f': [TAGS[16], TAGS[17], TAGS[18], TAGS[19]]
                     },
                '6': {
                      b'1': [TAGS[12]],
                      b'2': [TAGS[13]],
                      b'3': [TAGS[12], TAGS[13]],
                      b'4': [TAGS[14]],
                      b'5': [TAGS[12], TAGS[14]],
                      b'6': [TAGS[13], TAGS[14]],
                      b'7': [TAGS[12], TAGS[13], TAGS[14]]
                     },
                '7': {
                      b'1': [TAGS[8]],
                      b'2': [TAGS[9]],
                      b'3': [TAGS[8], TAGS[9]],
                      b'4': [TAGS[10]],
                      b'5': [TAGS[8], TAGS[10]],
                      b'6': [TAGS[25], TAGS[26]],
                      b'7': [TAGS[8], TAGS[9], TAGS[10]],
                      b'8': [TAGS[11]],
                      b'9': [TAGS[8], TAGS[11]],
                      b'a': [TAGS[9], TAGS[11]],
                      b'b': [TAGS[8], TAGS[9], TAGS[11]],
                      b'c': [TAGS[10], TAGS[11]],
                      b'd': [TAGS[8], TAGS[9], TAGS[11]],
                      b'e': [TAGS[8], TAGS[10], TAGS[11]],
                      b'f': [TAGS[8], TAGS[9], TAGS[10], TAGS[11]]
                     },
                '8': {
                      b'1': [TAGS[4]],
                      b'2': [TAGS[5]],
                      b'3': [TAGS[4], TAGS[5]],
                      b'4': [TAGS[6]],
                      b'5': [TAGS[4], TAGS[6]],
                      b'6': [TAGS[5], TAGS[6]],
                      b'7': [TAGS[4], TAGS[5], TAGS[6]]
                     },
                '9': {
                      b'1': [TAGS[0]],
                      b'2': [TAGS[1]],
                      b'3': [TAGS[0], TAGS[1]],
                      b'4': [TAGS[2]],
                      b'5': [TAGS[0], TAGS[2]],
                      b'6': [TAGS[25], TAGS[26]],
                      b'7': [TAGS[0], TAGS[1], TAGS[2]],
                      b'8': [TAGS[3]],
                      b'9': [TAGS[0], TAGS[3]],
                      b'a': [TAGS[1], TAGS[3]],
                      b'b': [TAGS[0], TAGS[1], TAGS[3]],
                      b'c': [TAGS[2], TAGS[3]],
                      b'd': [TAGS[0], TAGS[1], TAGS[3]],
                      b'e': [TAGS[0], TAGS[2], TAGS[3]],
                      b'f': [TAGS[0], TAGS[1], TAGS[2], TAGS[3]]
                }
              }

# Parameters do not currently contain the modulation routing parameters and are most likely 
# contributing to the bytes valued at 40 throughout the patch data. Will need to redo the entire 
# parameter dump file with the correct values. 

PARAMETERS = ('algorithm', 'c', 'a', 'b', 'harm', 'dtun', 'fdbk', 'mix', 'a_attack', 'a_decay', 
              'a_end','a_level', 'b_attack', 'b_decay', 'b_end', 'b_level','a_delay', 'a_trig', 
              'a_reset', 'phase_reset', 'b_delay', 'b_trig', 'b_reset', 'filt_attack', 'filt_dec', 
              'filt_sustain', 'filt_release', 'filt1_freq', 'filt1_reso', 'filt1_type', 'filt_env', 
              'filt2_base', 'filt2_width', 'amp_attack', 'amp_decay', 'amp_sustain', 'amp_release', 
              'drive','pan', 'vol', 'chorus', 'delay', 'reverb', 'amp_reset', 'lfo1_dest', 
              'lfo1_wave', 'lfo1_spd', 'lfo1_depth', 'lfo2_dest', 'lfo2_wave', 'lfo2_spd', 
              'lfo2_depth', 'lfo1_mult', 'lfo1_fade', 'lfo1_phase', 'lfo1_mode', 'lfo2_mult', 
              'lfo2_fade', 'lfo2_phase', 'lfo2_mode', 'arp_toggle', 'arp_speed', 'arp_range', 
              'arp_note_length', 'arp_offset01', 'arp_offset02', 'arp_offset03', 'arp_offset04', 
              'arp_offset05', 'arp_offset06', 'arp_offset07', 'arp_offset08', 'arp_offset09', 
              'arp_offset10', 'arp_offset11', 'arp_offset12', 'arp_offset13', 'arp_offset14', 
              'arp_offset15','arp_offset16', 'arp_length', 'arp_step01', 'arp_step02', 'arp_step03', 
              'arp_step04', 'arp_step05', 'arp_step06', 'arp_step07', 'arp_step08', 'arp_step09', 
              'arp_step10', 'arp_step11', 'arp_step12', 'arp_step13', 'arp_step14', 'arp_step15', 
              'arp_step16'
              )

# 'key_scale_a', 'key_scale_b1', 'key_scale_b2', 'filt_keytrack', 'velocity_vol', 'octave', 'pitchbend_dest1', 
# 'pitchbend_dest2', 'pitchbend_dest3', 'pitchbend_dest4', 'pitchbend_depth', 'pitchbend_amt1', 'pitchbend_amt2',
# 'pitchbend_amt3', 'pitchbend_amt4', 'velocity_dest1', 'velocity_dest2', 'velocity_dest3', 'velocity_dest4', 
# 'velocity_amt1', 'velocity_amt2', 'velocity_amt3'. 'velocity_amt4', 'modwheel_dest1', 'modwheel_dest2', 
# 'modwheel_dest3', 'modwheel_dest4', 'modwheel_amt1', 'modwheel_amt2', 'modwheel_amt3', 'modwheel_amt4', 
# 'breath_dest1', 'breath_dest2', 'breath_dest3', 'breath_dest4', 'breath_amt1', 'breath_amt2', 'breath_amt3', 
# 'breath_amt4', 'after_dest1', 'after_dest2', 'after_dest3', 'after_dest4', 'after_amt1', 'after_amt2', 
# 'after_amt3', 'after_amt4'

SYSEX_BEGIN = b'f0'+b'00'+b'20'+b'3c'+b'0d'+b'00'+b'53'+b'01'+b'01'


The binascii library is used to represent the binary syx file as hex digits. This may not be the most efficient method, but it has been essential in researching and deciphering the sysex data. This following code will break up the `SYSEX_FILE` and store it as an array. Each patch is then broken up into an array of all the individual bytes.

In [672]:
patches = []

with open(SYSEX_FILE, 'rb') as file:
    sysex = hexlify(file.read())
    for message in sysex.split(SYSEX_BEGIN):
        
        # filter out the the first split
        if len(message) > 20:
            patches.append(SYSEX_BEGIN + message)
        
    for patch in range(len(patches)):
        patches[patch] = [ patches[patch][i:i+2] for i in range(0, len(patches[patch]), 2)]

## Sysex Notes

The digitone does not send individual patches out by sysex. Instead, it can only send out an entire bank of patches as sysex data. It is a fairly simple task to split these up into their individual patches. Here is a sample patch with the binary data displayed next to it:

```text
 
0000  F0 00 20 3C 0D 00 53 01  01 03 78 3E 6F 3A 4E 00  |   <  S   x>o:N |
0010  00 00 00 01 00 00 00 08  4B 41 00 4C 52 4F 4E 20  |        KA LRON |
0020  50 41 00 44 00 00 00 00  00 00 00 00 00 02 00 70  |PA D           p|
0030  00 03 00 00 03 00 22 00  40 00 00 2A 00 00 00 00  |      " @  *    |
0040  00 00 00 00 11 00 00 00  01 00 08 00 00 3F 5B 40  |             ?[@|
0050  00 02 00 00 04 00 07 00  00 78 20 3E 4B 00 00 00  |         x >K   |
0060  00 21 00 00 01 00 00 00  00 00 00 3A 00 00 00 2F  | !         :   /|
0070  00 00 00 00 3A 00 00 00  2D 00 00 00 00 01 00 01  |    :   -       |
0080  00 00 00 00 01 00 01 00  00 00 00 40 00 40 00 40  |           @ @ @|
0090  00 40 00 00 40 00 40 00  40 00 02 40 00 01 00 53  | @  @ @ @  @   S|
00A0  5A 4C 10 00 72 1E 00 00  56 00 00 4F 00 4A 00 39  |ZL  r   V  O J 9|
00B0  00 76 00 00 08 00 20 00  60 00 00 37 22 00 00 5A  | v      `  7"  Z|
00C0  00 77 50 2A 51 73 00 00  00 00 00 01 00 00 00 00  | wP*Qs          |
00D0  00 00 00 00 00 00 00 00  00 00 00 00 00 00 00 00  |                |
00E0  00 00 00 00 02 00 00 00  00 00 00 00 00 00 00 00  |                |
00F0  00 00 00 00 00 00 00 00  00 00 00 00 00 00 00 00  |                |
0100  00 00 00 00 00 00 00 00  00 00 00 00 00 00 00 00  |                |
0110  00 00 00 00 00 00 09 00  0E 0F 60 7F 7F 00 00 00  |          `     |
0120  00 00 00 00 00 00 00 00  00 00 00 00 00 00 00 00  |                |
0130  00 00 00 00 00 00 02 00  00 00 00 00 00 00 00 00  |                |
0140  00 00 01 00 00 00 00 00  00 3A 60 4E 70 0C 27 26  |         :`Np '&|
0150  02 49 F7                                          | I               
```

This sample was taken from Sysex Librarian. It is recommened to use C6 from Elektron to get these sysex dumps.

### Packing

None of the data present in the patch dumps I have seen so far appears to be packed. The easy way to detect packed sysex is to look for bytes `0x81` or higher. From the monomachine sysex manual:

All dump packages on the Monomachine are packed. After that they are 7-bit encoded. To unpack check if bit 7 is set. If it is, the low 7 bits indicate that the next byte should be repeated this number of times in the resulting unpacked data structure. If bit 7 is not set the byte should just be stored in the unpacked data structure.

* An 8-bit data, XX, repeated 1 time is encoded `0x81` XX.
* A 7-bit data, YY, repeated 1 time is encoded YY.

#### How the 7 Bit Encoding is Generated

As a SysEx message only allows 7 bit data, 8 bit data blocks are converted as follows:

The first data contains the MSB of the following, up to, 7 data, data\[0..6\]. Bit 6 is the MSB bit of data\[0\] and bit 0 is the MSB of data\[6\]. 7 bytes encodes to 8 data. 8 bytes encodes to 10 data.

 __7 bit encoding example encoding bytes a, b, c:__
```
    
    Bit: 7   6  5  4  3  2  1  0
Data[0] [0 |a7|b7|c7|..|..|..|..]
Data[1] [0 |a6|a5|a4|a3|a2|a1|a0]
Data[2] [0 |b6|b5|b4|b3|b2|b1|b0]
Data[3] [0 |c6|c5|c4|c3|c2|c1|c0]
Data[4] ... 
```

### Prefix

Each message contains a prefix: `F0 00 20 3C 0D 00 53 01 01`. This contains the manufacturer and device information as well as some meta data about the patch. The empty bytes are placed as a buffer between the messages. 

`F0`, as always, signifies the beginning of a sysex message. `20 3C` are the two bytes that identify Elektron as the manufacturer. `0D` identifies the digitone as the device. `53` identifies that the dump contains a sound. The following `01 01` bytes are static and do not change in the patches. Further inspection is needed to see how these behave in other sysex messages from the digitone.

### Metadata

After the prefix, there is a string of data that I have yet to decipher. More analysis is need, These are the bytes from `0x0A` through `0x11`. Byte `0x0A` increases sequentially depending on the location of the patch in the bank. However, it stops counting after the first 128 patches in the bank. I cannot find another byte that continues this count. I believe this is a holdover from the same sysex system being used in other elektron devices. 

### Name

The name is stored in the bytes `0x18` through `0x28`. It can be viewed directly as a decoded string in python. However, it is currently printing more characters than displayed and causing formatting issues elsewhere. Specifically, note how the format method doesn't work correctly here.

In [669]:
for i in range(len(patches)):
    if i % 4 == 0:
        print()
        
    name = patches[i][int(0x18):int(0x1a)] + patches[i][int(0x1b):int(0x22)] + patches[i][int(0x23):(0x29)]
    name = unhexlify(b''.join(name)).decode('utf-8').strip('\x00').lstrip('\x00')
    print('{:20}'.format(name), end='')
    



DIGIT-ONE SM        CHAPPET DL          SQB TK              PLUCKY EE           
SIMPLE LEAD JM      SLIGHT DL           LIGHT BELL SM       RBASS HZ            
THAT ORGAN SM U     MISSPAD DL          JERRY BASS SM       GRUMLIG JM  JM      
GOLOW! DL           JAZZ ORG 1 MF       AUTOTEKNO BR  R     SAW BED SM          
WVVPZ ZB            LOUN HZ             FLUTE TK            CHOIRPAD ZB         
HOLD N GROW BR      EPIANO VGS JM       STRINGO EE          TREMBELL HZ         
DRIFT PLING SM      ACIENT STRM TK      AUTOCHORD SM        BASSJAM ZB          
PAD THAI EE         AYAKASHI HZ         SELAMETZ DL         ZETABASS ZB         
WITHOUT DL          SOFT KEYS EE        VIBRA16 SM          TX BASS 1 MF        
PULSES SM           BRASS1 JM  JM       RAVER SM            PV LEAD HZ          
BONGBANG DL         LOFI SAW SM         DOMEDAG JM          UBASS 1 HZ          
ALLOY BR            GRIMY SM S          GLITCH PAD BR       HOLD N SEE DL       
REEDY SM            BIG BRA

### Tags

I created a bank of patches that are all exactly the same except for the tags. I then exported those patches as a single sysex file and made the following observations:

In [588]:
print('LOC:\t 12 13 14 15 16 17')
print('--------------------------')
print()
message_counter = 0 

for patch in patches:

    tag = patch[int(0x12):int(0x18)]
    print('{}:\t {}'.format(TAGS[message_counter], 
                            b' '.join(tag).decode('utf-8')))
    message_counter += 1

    if message_counter % 8 == 0:
        print()

LOC:	 12 13 14 15 16 17
--------------------------

KICK:	 00 01 00 00 00 01
SNAR:	 00 01 00 00 00 02
DEEP:	 00 01 00 00 00 04
BRAS:	 00 01 00 00 00 08
STRI:	 00 01 00 00 00 10
PERC:	 00 01 00 00 00 20
HHAT:	 00 01 00 00 00 40
CYMB:	 04 01 00 00 00 00

EVOL:	 00 01 00 00 01 00
EXPR:	 00 01 00 00 02 00
BASS:	 00 01 00 00 04 00
LEAD:	 00 01 00 00 08 00
PAD:	 00 01 00 00 10 00
TXTR:	 00 01 00 00 20 00
CRD:	 00 01 00 00 40 00
SFX:	 08 01 00 00 00 00

ARP:	 00 01 00 01 00 00
METL:	 00 01 00 02 00 00
ACOU:	 00 01 00 04 00 00
ATMO:	 00 01 00 08 00 00
NOIS:	 00 01 00 10 00 00
GLCH:	 00 01 00 20 00 00
HARD:	 00 01 00 40 00 00
SOFT:	 10 01 00 00 00 00

DARK:	 00 01 01 00 00 00
BRGT:	 00 01 02 00 00 00
UNTG:	 00 01 04 00 00 00
EPIC:	 00 01 08 00 00 00
FAIL:	 00 01 10 00 00 00
LOOP:	 00 01 20 00 00 00
MINE:	 00 01 40 00 00 00
FAV:	 20 01 00 00 00 00



The data scheme for the tags seems immediately obvious from this run. Analyzing the data, it seems like the tags are stored as nibbles (apparently, that's really what you call half a byte). This makes sense as there are four tags per nibble. Take a look at the first eight patches:
```

LOC:	 12 13 14 15 16 17
--------------------------

KICK:	 00 01 00 00 00 01
SNAR:	 00 01 00 00 00 02
DEEP:	 00 01 00 00 00 04
BRAS:	 00 01 00 00 00 08
STRI:	 00 01 00 00 00 10
PERC:	 00 01 00 00 00 20
HHAT:	 00 01 00 00 00 40
CYMB:	 04 01 00 00 00 00
```

The first four eat up the lower nibble of byte `0x17`, meaning we can represent all four tags being active by setting this nibble to `F`. Notice how the higher nibble in the byte only stores up three of the tags. The higher nibble cannot be greater than `7`, as this would indicate a packed data package.

Now, let's combine the code above to get the tags of all the patches!

__NOTE__: While this currently works, it's a very ugly way to get it done. I'm sure there is a more pythonic way to iterate through these and get it done. This needs to be analyzed in depth to make sure that the tagger script is working. A cursory glance seems to indiate that it is working fine, but more analysis is needed.

In [593]:
for i in range(32):
    if i % 8 == 0:
        print()

    # Convert the tag bytes to a string for display and iteration
    tag_msg = [ patches[i][int(0x12)] ] + patches[i][int(0x14):int(0x18)]
    tag_msg = b''.join(tag_msg).decode('utf-8')    
    print('Patch #{:04d}:\t {}\t '.format(i, ' '.join(tag_msg[i:i+2] for i in range(0, len(tag_msg), 2))), end='')
    
    tags = []
    for j in range(len(tag_msg)):
        if tag_msg[j] in TAGS_LOOKUP[str(j)].keys():
            tags = tags + TAGS_LOOKUP[str(j)][tag_msg[j]]
    
    print(' '.join(tags))


Patch #0000:	 00 08 00 00 00	 EPIC
Patch #0001:	 10 00 08 20 00	 SOFT ATMO TXTR
Patch #0002:	 00 00 00 04 00	 BASS
Patch #0003:	 00 00 00 02 20	 EXPR PERC
Patch #0004:	 00 00 00 00 00	 
Patch #0005:	 00 00 08 08 00	 ATMO LEAD
Patch #0006:	 10 02 06 00 00	 SOFT BRGT BRGT UNTG
Patch #0007:	 08 01 28 04 00	 SFX DARK GLCH ATMO BASS

Patch #0008:	 00 0c 01 04 00	 UNTG EPIC ARP BASS
Patch #0009:	 00 00 00 30 00	 PAD TXTR
Patch #0010:	 00 04 04 04 00	 UNTG ACOU BASS
Patch #0011:	 00 00 00 00 00	 
Patch #0012:	 10 02 00 08 00	 SOFT BRGT LEAD
Patch #0013:	 10 04 04 00 00	 SOFT UNTG ACOU
Patch #0014:	 00 23 50 00 01	 LOOP DARK BRGT NOIS HARD KICK
Patch #0015:	 00 05 00 02 00	 DARK UNTG EXPR

Patch #0016:	 09 20 51 04 00	 LOOP NOIS HARD ARP BASS
Patch #0017:	 10 04 0c 00 00	 SOFT UNTG ACOU ATMO
Patch #0018:	 00 00 04 00 00	 ACOU
Patch #0019:	 10 00 00 52 00	 SOFT PAD CRD EXPR
Patch #0020:	 08 00 21 30 00	 SFX GLCH ARP PAD TXTR
Patch #0021:	 00 00 00 00 00	 
Patch #0022:	 00 02 04 00 00	 BRGT ACO

### Parameters

To decipher where the patches are stored, I created a patch with every parameter turned down to 0. From there, I created a new patch for each parameter turned to its maximum value. The following script compares all of the parameter patches and identifies the locations for each parameter.

The data block that contains all of the patch data comes after the bytes reseverd for the name space. They are the bytes `0x29` through `0x14e`.

In [680]:
parameter_lookup = {}

for i in range(len(patches)):
    if i == 0:
        init_values = patches[i][int(0x29):int(0x14e)]
    else:
        data = patches[i][int(0x29):int(0x14e)]
        for datum in range(len(data)):
            if PARAMETERS[i - 1] not in parameter_lookup.keys():
                parameter_lookup[PARAMETERS[i - 1]] = []
                
            if data[datum] != init_values[datum]:               
                parameter_lookup[PARAMETERS[i - 1]].append(hex(datum).strip())

print('{:20}'.format('-'*51))
print('| {:2} | {:20} | {:20}|'.format('#','Parameter', 'Location') )
print('{:20}'.format('-'*51))

# Do not display trig parameters
i = 1
for parameter in PARAMETERS:
    print('| {:02} | {:20} | {:20}|'.format(i, parameter, ' '.join(parameter_lookup[parameter])))
    i += 1

print('{:20}'.format('-'*51))

print()
print(parameter_lookup)

---------------------------------------------------
| #  | Parameter            | Location            |
---------------------------------------------------
| 01 | algorithm            | 0x28                |
| 02 | c                    | 0x2b                |
| 03 | a                    | 0x2d                |
| 04 | b                    | 0x29 0x2f 0x30      |
| 05 | harm                 | 0x32                |
| 06 | dtun                 | 0x34                |
| 07 | fdbk                 | 0x36                |
| 08 | mix                  | 0x38                |
| 09 | a_attack             | 0x3f                |
| 10 | a_decay              | 0x42                |
| 11 | a_end                | 0x44                |
| 12 | a_level              | 0x46                |
| 13 | b_attack             | 0x48                |
| 14 | b_decay              | 0x4b                |
| 15 | b_end                | 0x4d                |
| 16 | b_level              | 0x4f                |
| 17 | a_del

### Notes

Several of the parameters access several bytes to store their data. These are:

```
---------------------------------------------
|Parameter            | Location            |
---------------------------------------------
|b                    | 0x59 0x58 0x52      |
|lfo1_spd             | 0x2e 0x2d 0x2a      |
|lfo1_depth           | 0x4e 0x4d 0x4a      |
|lfo2_spd             | 0x30 0x2f 0x2a      |
|lfo2_depth           | 0x50 0x4f 0x4a      |
---------------------------------------------
```

I'm a little surprised not to see `harm` on this list, but it do what it be. The final byte in each of these (except b) sequences are the same, suggesting it may not be significant. It may also hold a separate value if both are on, which should be inspected. The arpeggio information will be especially tricky to decipher. I believe that `0x12d` may not be used in the step toggle and arp length parameters. More research is needed here. 

A dictionary has been created with the above information:

```
{'algorithm': '0x28', 'c': '0x2b', 'a': '0x2d', 'b': '0x29 0x2f 0x30', 'harm': '0x32', 'dtun': '0x34', 'fdbk': '0x36', 'mix': '0x38', 'a_attack': '0x3f', 'a_decay': '0x42', 'a_end': '0x44', 'a_level': '0x46', 'b_attack': '0x48', 'b_decay': '0x4b', 'b_end': '0x4d', 'b_level': '0x4f', 'a_delay': '0x52', 'a_trig': '0x54', 'a_reset': '0x56', 'phase_reset': '0x3b', 'b_delay': '0x58', 'b_trig': '0x5b', 'b_reset': '0x5d', 'filt_attack': '0x7d', 'filt_dec': '0x7f', 'filt_sustain': '0x82', 'filt_release': '0x84', 'filt1_freq': '0x76', 'filt1_reso': '0x78', 'filt1_type': '0x74', 'filt_env': '0x7b', 'filt2_base': '0x86', 'filt2_width': '0x88', 'amp_attack': '0x8b', 'amp_decay': '0x8d', 'amp_sustain': '0x8f', 'amp_release': '0x92', 'drive': '0x94', 'pan': '0x96', 'vol': '0x98', 'chorus': '0x9f', 'delay': '0x9d', 'reverb': '0x9b', 'amp_reset': '0xa2', 'lfo1_dest': '0x12', 'lfo1_wave': '0x16', 'lfo1_spd': '0x1 0x4 0x5', 'lfo1_depth': '0x21 0x24 0x25', 'lfo2_dest': '0x14', 'lfo2_wave': '0x18', 'lfo2_spd': '0x1 0x6 0x7', 'lfo2_depth': '0x21 0x26 0x27', 'lfo1_mult': '0x8', 'lfo1_fade': '0xd', 'lfo1_phase': '0x1b', 'lfo1_mode': '0x1f', 'lfo2_mult': '0xb', 'lfo2_fade': '0xf', 'lfo2_phase': '0x1d', 'lfo2_mode': '0x22', 'arp_toggle': '0xec', 'arp_speed': '0xed', 'arp_range': '0xee', 'arp_note_length': '0xef', 'arp_offset01': '0xf1 0xf4', 'arp_offset02': '0xf1 0xf5', 'arp_offset03': '0xf1 0xf6', 'arp_offset04': '0xf1 0xf7', 'arp_offset05': '0xf1 0xf8', 'arp_offset06': '0xf9 0xfa', 'arp_offset07': '0xf9 0xfb', 'arp_offset08': '0xf9 0xfc', 'arp_offset09': '0xf9 0xfd', 'arp_offset10': '0xf9 0xfe', 'arp_offset11': '0xf9 0xff', 'arp_offset12': '0xf9 0x100', 'arp_offset13': '0x101 0x102', 'arp_offset14': '0x101 0x103', 'arp_offset15': '0x101 0x104', 'arp_offset16': '0x101 0x104 0x105', 'arp_length': '0xf0 0x104', 'arp_step01': '0xf3 0x104', 'arp_step02': '0xf3 0x104', 'arp_step03': '0xf3 0x104', 'arp_step04': '0xf3 0x104', 'arp_step05': '0xf3 0x104', 'arp_step06': '0xf3 0x104', 'arp_step07': '0xf3 0x104', 'arp_step08': '0xf1 0x104', 'arp_step09': '0xf2 0x104', 'arp_step10': '0xf2 0x104', 'arp_step11': '0xf2 0x104', 'arp_step12': '0xf2 0x104', 'arp_step13': '0xf2 0x104', 'arp_step14': '0xf2 0x104', 'arp_step15': '0xf2 0x104', 'arp_step16': '0xf1 0x104'}
```

## End Message

The monomachine and machinedrum manuals give hints towards the checksum scheme of the digitone. This has not been verified, but the scheme should be consistent.

The last 5 bytes of a message are the same.

* Checksum (bit 7 - 13)
* Checksum (bit 0 - 6)
* Message Length (bit 7 - 13)
* Message Length (bit 0 - 6)
* Endy of SysEx

In the state example above, it would equate to this string of bytes at the end: `27 26 02 49 F7`. 

### Checksum

The checksum adds all of the data bytes together and stores the last 14 bits of that number. While I label many of these arrays of bytes different things throughout this document, the checksum counts all the bytes between `0x0a` and `0x14e`. 

### Message Length

The last two bytes before the EOM byte store the message length. The message includes the EOM part, but not the prefix. 

In [632]:
for i in range(8):
    
    eom = patches[i][int(0x14e):]
    checksum = eom[:2]
    
    print("EOM: {}".format(b' '.join(eom).decode('utf-8')))

    data_total = 0
    check = patches[i][int(0x0a):int(0x14e)]
    for datum in check:
            data_total += int(datum, 16)

    data_total = bin(data_total)[2:]
    missing_bytes = 14 - len(data_total)
    if missing_bytes > 0:
        data_total = ('0'*missing_bytes) + data_total
    else: 
        data_total = data_total[:14]
        
    checksum_msb = hex(int(data_total[:7], 2))
    checksum_lsb = hex(int(data_total[7:], 2))
    
    message_length = bin(len(patches[i][int(0x0a):]))[2:]
    missing_bytes = 14 - len(message_length)
    if missing_bytes > 0:
        message_length = ('0'*missing_bytes) + message_length
    else:
        message_length = message_length[:14]
    
    message_msb = hex(int(message_length[:7], 2))
    message_lsb = hex(int(message_length[7:], 2))


    print("Checksum: {} {} \nMessage Length: {} {}".format(checksum_msb, checksum_lsb,
                                                          message_msb, message_lsb))
    print('-'*30,'\n')
    
#     print(hex((len(data)*2) - 1 ))
    

EOM: 2b 5e 02 49 f7
Checksum: 0x2b 0x5e 
Message Length: 0x2 0x49
------------------------------ 

EOM: 28 48 02 49 f7
Checksum: 0x28 0x48 
Message Length: 0x2 0x49
------------------------------ 

EOM: 23 73 02 49 f7
Checksum: 0x23 0x73 
Message Length: 0x2 0x49
------------------------------ 

EOM: 27 26 02 49 f7
Checksum: 0x27 0x26 
Message Length: 0x2 0x49
------------------------------ 

EOM: 2f 31 02 49 f7
Checksum: 0x2f 0x31 
Message Length: 0x2 0x49
------------------------------ 

EOM: 29 62 02 49 f7
Checksum: 0x29 0x62 
Message Length: 0x2 0x49
------------------------------ 

EOM: 26 45 02 49 f7
Checksum: 0x26 0x45 
Message Length: 0x2 0x49
------------------------------ 

EOM: 24 43 02 49 f7
Checksum: 0x24 0x43 
Message Length: 0x2 0x49
------------------------------ 



# Utilities

These scripts help with working with the files.

### Quick Merge Script

In [ ]:
# quick merge script

file1 = open('factory.syx', 'rb')
file2 = open('big_space.syx', 'rb')
file3 = open('d_tones.syx', 'rb')
file4 = open('haunted_hearts.syx', 'rb')
file5 = open('rtfm.syx', 'rb')

# final = open('all_patches.syx', 'wb')

# data = file1.read() + file2.read() + file3.read() + file4.read() + file5.read()
# final.write(data)
# final.close()

test = open('all_patches.syx', 'rb').read()

print(test)

Split a message into all of it's parts!

In [ ]:
for patch in patches:
    
    prefix = patch[:int(0x0A)]
    unspec = patch[int(0x0a):int(0x12)]
    tag = patch[int(0x12):int(0x18)]
    name = patch[int(0x18):int(0x29)]
    data = patch[int(0x29):int(0x14e)]
    eom = patch[int(0x14e):]
    checksum_msb = eom[0]
    checksum_lsb = eom[1]
    checksum = eom[0:2]
    
    print("Prefix: {}".format(prefix))
    print("Unspecified: {}".format(unspec))
    print("Tag: {}".format(tag))
    print("Name: {}".format(name))
    print("Data: {}".format(data))
    print("EOM: {}".format(eom))
    print()